# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 8:30AM,242792,10,0085907661,ISDIN Corporation,Released
1,Jul 27 2022 8:30AM,242792,10,0085907893,ISDIN Corporation,Released
2,Jul 27 2022 8:30AM,242792,10,0085907888,ISDIN Corporation,Released
3,Jul 27 2022 8:30AM,242792,10,0085907989,ISDIN Corporation,Released
4,Jul 27 2022 8:30AM,242792,10,0085907990,ISDIN Corporation,Released
5,Jul 27 2022 8:30AM,242792,10,0085908096,ISDIN Corporation,Released
6,Jul 27 2022 8:30AM,242792,10,0085908123,ISDIN Corporation,Released
7,Jul 27 2022 8:30AM,242792,10,0085908151,ISDIN Corporation,Released
8,Jul 27 2022 8:30AM,242792,10,0085908157,ISDIN Corporation,Released
9,Jul 27 2022 8:30AM,242792,10,0085908183,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,242820,Released,2
39,242821,Released,14
40,242822,Released,1
41,242823,Released,9
42,242824,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242820,NaN,NaN,2.0
242821,NaN,NaN,14.0
242822,NaN,NaN,1.0
242823,NaN,NaN,9.0
242824,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242718,36.0,2.0,1.0
242720,0.0,4.0,46.0
242721,13.0,1.0,0.0
242723,0.0,5.0,26.0
242733,1.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242718,36,2,1
242720,0,4,46
242721,13,1,0
242723,0,5,26
242733,1,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242718,36,2,1
1,242720,0,4,46
2,242721,13,1,0
3,242723,0,5,26
4,242733,1,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242718,36,2,1
1,242720,,4,46
2,242721,13,1,
3,242723,,5,26
4,242733,1,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation
18,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance
19,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation
28,Jul 27 2022 8:11AM,242822,10,Emerginnova
29,Jul 27 2022 8:03AM,242821,10,"Methapharm, Inc."
43,Jul 27 2022 8:02AM,242820,10,"Methapharm, Inc."
45,Jul 27 2022 7:51AM,242819,20,Alumier Labs Inc.
73,Jul 27 2022 7:22AM,242818,19,"AdvaGen Pharma, Ltd"
94,Jul 26 2022 5:45PM,242817,21,"NBTY Global, Inc."
95,Jul 26 2022 5:44PM,242816,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,,,18
1,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,,,1
2,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,,,9
3,Jul 27 2022 8:11AM,242822,10,Emerginnova,,,1
4,Jul 27 2022 8:03AM,242821,10,"Methapharm, Inc.",,,14
5,Jul 27 2022 8:02AM,242820,10,"Methapharm, Inc.",,,2
6,Jul 27 2022 7:51AM,242819,20,Alumier Labs Inc.,,,28
7,Jul 27 2022 7:22AM,242818,19,"AdvaGen Pharma, Ltd",,,21
8,Jul 26 2022 5:45PM,242817,21,"NBTY Global, Inc.",,,1
9,Jul 26 2022 5:44PM,242816,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18,,
1,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,1,,
2,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,9,,
3,Jul 27 2022 8:11AM,242822,10,Emerginnova,1,,
4,Jul 27 2022 8:03AM,242821,10,"Methapharm, Inc.",14,,
5,Jul 27 2022 8:02AM,242820,10,"Methapharm, Inc.",2,,
6,Jul 27 2022 7:51AM,242819,20,Alumier Labs Inc.,28,,
7,Jul 27 2022 7:22AM,242818,19,"AdvaGen Pharma, Ltd",21,,
8,Jul 26 2022 5:45PM,242817,21,"NBTY Global, Inc.",1,,
9,Jul 26 2022 5:44PM,242816,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18,,
1,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,1,,
2,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,9,,
3,Jul 27 2022 8:11AM,242822,10,Emerginnova,1,,
4,Jul 27 2022 8:03AM,242821,10,"Methapharm, Inc.",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18.0,NaN,NaN
1,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,1.0,NaN,NaN
2,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,9.0,NaN,NaN
3,Jul 27 2022 8:11AM,242822,10,Emerginnova,1.0,NaN,NaN
4,Jul 27 2022 8:03AM,242821,10,"Methapharm, Inc.",14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18.0,0.0,0.0
1,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,1.0,0.0,0.0
2,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,9.0,0.0,0.0
3,Jul 27 2022 8:11AM,242822,10,Emerginnova,1.0,0.0,0.0
4,Jul 27 2022 8:03AM,242821,10,"Methapharm, Inc.",14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2913662,73.0,0.0,12.0
15,242733,1.0,0.0,1.0
16,971053,4.0,2.0,26.0
19,1942386,29.0,0.0,0.0
20,1699263,116.0,60.0,38.0
21,485633,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2913662,73.0,0.0,12.0
1,15,242733,1.0,0.0,1.0
2,16,971053,4.0,2.0,26.0
3,19,1942386,29.0,0.0,0.0
4,20,1699263,116.0,60.0,38.0
5,21,485633,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73.0,0.0,12.0
1,15,1.0,0.0,1.0
2,16,4.0,2.0,26.0
3,19,29.0,0.0,0.0
4,20,116.0,60.0,38.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,15,Released,1.0
2,16,Released,4.0
3,19,Released,29.0
4,20,Released,116.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,12.0,1.0,26.0,0.0,38.0,0.0
Executing,0.0,0.0,2.0,0.0,60.0,0.0
Released,73.0,1.0,4.0,29.0,116.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,12.0,1.0,26.0,0.0,38.0,0.0
1,Executing,0.0,0.0,2.0,0.0,60.0,0.0
2,Released,73.0,1.0,4.0,29.0,116.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,12.0,1.0,26.0,0.0,38.0,0.0
1,Executing,0.0,0.0,2.0,0.0,60.0,0.0
2,Released,73.0,1.0,4.0,29.0,116.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()